# Lab 3: Operators and Hooks - Executing Tasks with Airflow Operators

## 🎯 Objectives
- Understand the role of Operators in Airflow
- Use common operators: BashOperator, PythonOperator, SQLExecuteQueryOperator
- Learn how to use Hooks to connect to external systems
- Compare @task decorator vs Operators
- Create data pipeline with multiple operator types

## 📋 Prerequisites
- Completed Lab 1: Airflow Basics
- Completed Lab 2: DAGs and Tasks
- Airflow cluster is running
- Basic Python and SQL knowledge

## 🏗️ Operators Overview
Operators are templates for tasks in Airflow. Each operator defines a specific type of work:
- **BashOperator**: Run bash commands
- **PythonOperator**: Run Python functions
- **SQLExecuteQueryOperator**: Execute SQL queries
- **EmailOperator**: Send emails
- **HttpOperator**: Make HTTP requests
- And many other operators from providers


## 1. Import Libraries and Setup


In [ ]:
# Import Airflow operators and hooks
from airflow.sdk import DAG
from airflow.providers.standard.operators.bash import BashOperator
from airflow.providers.standard.operators.python import PythonOperator
from airflow.providers.common.sql.operators.sql import SQLExecuteQueryOperator
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.hooks.base import BaseHook

import pendulum
from datetime import datetime, timedelta
import os

print("✅ Airflow operators và hooks imported successfully!")


## 2. BashOperator - Run Shell Commands

BashOperator allows running bash commands in tasks. Very useful for:
- File operations
- System commands
- Script execution
- Data processing with command-line tools


In [ ]:
# DAG with BashOperator
@dag(
    dag_id="bash_operator_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["operators", "bash"],
)
def bash_operator_dag():
    """
    ### BashOperator Example
    Sample DAG using BashOperator to run shell commands.
    """
    
    # Task 1: Simple bash command
    print_date = BashOperator(
        task_id="print_date",
        bash_command="date",
    )
    
    # Task 2: Multi-line bash command
    create_directory = BashOperator(
        task_id="create_directory",
        bash_command="""
        mkdir -p /tmp/airflow_test
        echo "Directory created at $(date)" > /tmp/airflow_test/created.txt
        cat /tmp/airflow_test/created.txt
        """,
    )
    
    # Task 3: Bash with environment variables
    print_env = BashOperator(
        task_id="print_env",
        bash_command="echo 'Python path: $PYTHONPATH' && echo 'User: $USER'",
        env={"CUSTOM_VAR": "custom_value"},
    )
    
    # Task 4: Bash with Jinja templating
    templated_command = BashOperator(
        task_id="templated_command",
        bash_command="""
        echo "Execution date: {{ ds }}"
        echo "Next execution date: {{ next_ds }}"
        echo "Previous execution date: {{ prev_ds }}"
        echo "DAG run ID: {{ dag_run.dag_id }}"
        """,
    )
    
    # Define dependencies
    print_date >> create_directory >> print_env >> templated_command

# Create DAG
bash_dag = bash_operator_dag()

print("✅ BashOperator DAG created!")
print(f"Tasks: {[task.task_id for task in bash_dag.tasks]}")


## 3. PythonOperator - Run Python Functions

PythonOperator allows running Python functions as tasks. Note:
- Function must be defined at top-level or importable
- Can pass arguments via `op_args` and `op_kwargs`
- Context variables are available in function


In [ ]:
# Helper functions for PythonOperator
def print_context(**context):
    """Print Airflow context variables"""
    print("=" * 60)
    print("Airflow Context Variables:")
    print(f"  Execution Date: {context['ds']}")
    print(f"  Next Execution Date: {context['next_ds']}")
    print(f"  Previous Execution Date: {context['prev_ds']}")
    print(f"  DAG Run ID: {context['dag_run'].dag_id}")
    print(f"  Task Instance: {context['task_instance'].task_id}")
    print("=" * 60)
    return "Context printed successfully"

def process_data(records: int, multiplier: float = 1.5):
    """Process data with parameters"""
    print(f"Processing {records} records with multiplier {multiplier}")
    result = records * multiplier
    print(f"Result: {result}")
    return result

def generate_report(**context):
    """Generate report with context"""
    execution_date = context['ds']
    dag_id = context['dag_run'].dag_id
    
    report = f"""
    Report Generated
    ================
    DAG: {dag_id}
    Execution Date: {execution_date}
    Status: Success
    """
    print(report)
    return report

# DAG with PythonOperator
@dag(
    dag_id="python_operator_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["operators", "python"],
)
def python_operator_dag():
    """
    ### PythonOperator Example
    Sample DAG using PythonOperator to run Python functions.
    """
    
    # Task 1: Simple Python function
    print_context_task = PythonOperator(
        task_id="print_context",
        python_callable=print_context,
    )
    
    # Task 2: Python function with arguments
    process_data_task = PythonOperator(
        task_id="process_data",
        python_callable=process_data,
        op_args=[100],  # Positional arguments
        op_kwargs={"multiplier": 2.0},  # Keyword arguments
    )
    
    # Task 3: Python function with context
    generate_report_task = PythonOperator(
        task_id="generate_report",
        python_callable=generate_report,
    )
    
    # Define dependencies
    print_context_task >> process_data_task >> generate_report_task

# Create DAG
python_dag = python_operator_dag()

print("✅ PythonOperator DAG created!")
print(f"Tasks: {[task.task_id for task in python_dag.tasks]}")


In [ ]:
# DAG với SQLExecuteQueryOperator
# Note: Cần setup PostgreSQL connection trong Airflow UI trước
# Connection ID: postgres_default
# Connection Type: Postgres
# Host: postgres
# Schema: airflow
# Login: airflow
# Password: airflow
# Port: 5432

@dag(
    dag_id="sql_operator_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["operators", "sql"],
)
def sql_operator_dag():
    """
    ### SQLExecuteQueryOperator Example
    DAG mẫu sử dụng SQLExecuteQueryOperator để thực thi SQL queries.
    
    Note: Cần setup PostgreSQL connection trong Airflow UI:
    - Connection ID: postgres_default
    - Connection Type: Postgres
    - Host: postgres
    - Schema: airflow
    - Login: airflow
    - Password: airflow
    - Port: 5432
    """
    
    # Task 1: Create table
    create_table = SQLExecuteQueryOperator(
        task_id="create_table",
        conn_id="postgres_default",
        sql="""
        CREATE TABLE IF NOT EXISTS airflow_test_table (
            id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            value INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """,
    )
    
    # Task 2: Insert data với Jinja templating
    insert_data = SQLExecuteQueryOperator(
        task_id="insert_data",
        conn_id="postgres_default",
        sql="""
        INSERT INTO airflow_test_table (name, value)
        VALUES 
            ('{{ ds }}', 100),
            ('test_record', 200),
            ('another_record', 300);
        """,
    )
    
    # Task 3: Query data
    query_data = SQLExecuteQueryOperator(
        task_id="query_data",
        conn_id="postgres_default",
        sql="""
        SELECT 
            COUNT(*) as total_records,
            SUM(value) as total_value,
            AVG(value) as avg_value
        FROM airflow_test_table;
        """,
    )
    
    # Task 4: Cleanup (optional)
    drop_table = SQLExecuteQueryOperator(
        task_id="drop_table",
        conn_id="postgres_default",
        sql="DROP TABLE IF EXISTS airflow_test_table;",
    )
    
    # Define dependencies
    create_table >> insert_data >> query_data
    # Uncomment để cleanup:
    # query_data >> drop_table

# Create DAG
sql_dag = sql_operator_dag()

print("✅ SQLExecuteQueryOperator DAG created!")
print(f"Tasks: {[task.task_id for task in sql_dag.tasks]}")
print("\n💡 Note: Setup PostgreSQL connection in Airflow UI before running this DAG")


In [ ]:
# DAG với Hooks
@dag(
    dag_id="hooks_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["hooks", "example"],
)
def hooks_dag():
    """
    ### Hooks Example
    DAG mẫu sử dụng Hooks để kết nối với external systems.
    """
    
    def use_postgres_hook(**context):
        """Sử dụng PostgresHook để kết nối database"""
        # Get hook từ connection ID
        hook = PostgresHook(postgres_conn_id="postgres_default")
        
        # Execute query
        records = hook.get_records("""
            SELECT COUNT(*) FROM information_schema.tables 
            WHERE table_schema = 'public'
        """)
        
        print(f"Number of tables in public schema: {records[0][0]}")
        
        # Get pandas dataframe
        df = hook.get_pandas_df("SELECT version();")
        print(f"PostgreSQL version: {df.iloc[0, 0]}")
        
        return records[0][0]
    
    def test_connection(**context):
        """Test database connection"""
        hook = PostgresHook(postgres_conn_id="postgres_default")
        
        # Test connection
        try:
            conn = hook.get_conn()
            print("✅ Database connection successful!")
            print(f"Connection info: {conn.info}")
            conn.close()
            return "Connection OK"
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            raise
    
    # Tasks sử dụng hooks
    test_conn_task = PythonOperator(
        task_id="test_connection",
        python_callable=test_connection,
    )
    
    use_hook_task = PythonOperator(
        task_id="use_postgres_hook",
        python_callable=use_postgres_hook,
    )
    
    # Define dependencies
    test_conn_task >> use_hook_task

# Create DAG
hooks_dag_instance = hooks_dag()

print("✅ Hooks DAG created!")
print(f"Tasks: {[task.task_id for task in hooks_dag_instance.tasks]}")


## 6. So sánh @task Decorator vs Operators

Airflow cung cấp 2 cách để tạo tasks:
1. **@task decorator** (Task SDK - Modern approach)
2. **Operators** (Traditional approach)

Hãy so sánh ưu/nhược điểm của mỗi cách:


In [ ]:
# So sánh @task decorator vs PythonOperator

from airflow.sdk import task

@dag(
    dag_id="comparison_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["comparison", "example"],
)
def comparison_dag():
    """
    ### Comparison: @task vs Operators
    So sánh cách sử dụng @task decorator và Operators.
    """
    
    # Cách 1: Sử dụng @task decorator (Modern - Task SDK)
    @task
    def task_sdk_example(data: dict):
        """Task SDK approach - Clean và Pythonic"""
        result = data.get("value", 0) * 2
        print(f"Task SDK result: {result}")
        return result
    
    # Cách 2: Sử dụng PythonOperator (Traditional)
    def python_operator_function(**context):
        """PythonOperator approach - More explicit"""
        data = {"value": 100}
        result = data.get("value", 0) * 2
        print(f"PythonOperator result: {result}")
        return result
    
    python_operator_task = PythonOperator(
        task_id="python_operator_task",
        python_callable=python_operator_function,
    )
    
    # Cách 3: Sử dụng BashOperator (Không có @task tương đương)
    bash_task = BashOperator(
        task_id="bash_task",
        bash_command="echo 'BashOperator - No @task equivalent'",
    )
    
    # Task SDK với data passing
    initial_data = {"value": 50}
    task_sdk_result = task_sdk_example(initial_data)
    
    # Define dependencies
    bash_task >> python_operator_task >> task_sdk_result

# Create DAG
comparison_dag_instance = comparison_dag()

print("✅ Comparison DAG created!")
print("\n📊 Comparison Summary:")
print("=" * 60)
print("@task Decorator:")
print("  ✅ Clean, Pythonic syntax")
print("  ✅ Automatic XCom handling")
print("  ✅ Type hints support")
print("  ✅ Easier data passing")
print("  ❌ Limited to Python functions")
print("\nOperators:")
print("  ✅ Support many types (Bash, SQL, HTTP, etc.)")
print("  ✅ More explicit và configurable")
print("  ✅ Better for complex operations")
print("  ❌ More verbose syntax")
print("  ❌ Manual XCom handling")
print("=" * 60)


## 7. Mixed Pipeline - Kết hợp nhiều Operators

Trong thực tế, pipelines thường sử dụng nhiều loại operators khác nhau.
Hãy tạo một pipeline hoàn chỉnh với nhiều operators:


In [ ]:
# Mixed Pipeline với nhiều operators
@dag(
    dag_id="mixed_operators_pipeline",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["pipeline", "mixed-operators"],
)
def mixed_pipeline():
    """
    ### Mixed Operators Pipeline
    Pipeline mẫu kết hợp nhiều loại operators.
    """
    
    # Step 1: Bash - Prepare environment
    prepare_env = BashOperator(
        task_id="prepare_environment",
        bash_command="""
        mkdir -p /tmp/airflow_pipeline
        echo "Pipeline started at $(date)" > /tmp/airflow_pipeline/log.txt
        """,
    )
    
    # Step 2: Python - Extract data
    def extract_data(**context):
        """Extract data from source"""
        import json
        
        data = {
            "date": context['ds'],
            "records": [
                {"id": 1, "name": "Alice", "score": 95},
                {"id": 2, "name": "Bob", "score": 87},
                {"id": 3, "name": "Charlie", "score": 92},
            ]
        }
        
        # Save to file
        output_path = "/tmp/airflow_pipeline/data.json"
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"Extracted {len(data['records'])} records")
        return output_path
    
    extract_task = PythonOperator(
        task_id="extract_data",
        python_callable=extract_data,
    )
    
    # Step 3: Python - Transform data
    def transform_data(**context):
        """Transform extracted data"""
        import json
        
        # Get file path from previous task (via XCom)
        ti = context['ti']
        file_path = ti.xcom_pull(task_ids='extract_data')
        
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Calculate statistics
        scores = [record['score'] for record in data['records']]
        stats = {
            "total_records": len(scores),
            "average_score": sum(scores) / len(scores),
            "max_score": max(scores),
            "min_score": min(scores),
        }
        
        # Save transformed data
        output_path = "/tmp/airflow_pipeline/stats.json"
        with open(output_path, 'w') as f:
            json.dump(stats, f, indent=2)
        
        print(f"Transformed data: {stats}")
        return output_path
    
    transform_task = PythonOperator(
        task_id="transform_data",
        python_callable=transform_data,
    )
    
    # Step 4: Bash - Generate report
    generate_report = BashOperator(
        task_id="generate_report",
        bash_command="""
        echo "=== Pipeline Report ===" >> /tmp/airflow_pipeline/log.txt
        echo "Date: {{ ds }}" >> /tmp/airflow_pipeline/log.txt
        echo "Status: Success" >> /tmp/airflow_pipeline/log.txt
        cat /tmp/airflow_pipeline/log.txt
        """,
    )
    
    # Step 5: Python - Cleanup
    def cleanup(**context):
        """Cleanup temporary files"""
        import os
        
        files_to_remove = [
            "/tmp/airflow_pipeline/data.json",
            "/tmp/airflow_pipeline/stats.json",
        ]
        
        for file_path in files_to_remove:
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"Removed: {file_path}")
        
        print("Cleanup completed")
    
    cleanup_task = PythonOperator(
        task_id="cleanup",
        python_callable=cleanup,
        trigger_rule="all_done",  # Run even if previous tasks failed
    )
    
    # Define dependencies
    prepare_env >> extract_task >> transform_task >> generate_report >> cleanup_task

# Create DAG
mixed_pipeline_dag = mixed_pipeline()

print("✅ Mixed Operators Pipeline DAG created!")
print(f"Tasks: {[task.task_id for task in mixed_pipeline_dag.tasks]}")


## 8. Best Practices và Tips

### ✅ Best Practices:

1. **Chọn đúng Operator:**
   - BashOperator: Cho shell commands và scripts
   - PythonOperator: Cho Python logic phức tạp
   - SQLExecuteQueryOperator: Cho database operations
   - @task decorator: Cho Python functions đơn giản

2. **Connection Management:**
   - Luôn sử dụng Airflow Connections thay vì hardcode credentials
   - Test connections trước khi deploy
   - Sử dụng Hooks để reuse connections

3. **Error Handling:**
   - Set retries và retry_delay phù hợp
   - Sử dụng trigger_rule để handle failures
   - Log errors đầy đủ

4. **Performance:**
   - Tránh heavy operations trong PythonOperator
   - Sử dụng appropriate executors
   - Cache kết quả khi có thể

5. **Code Organization:**
   - Đặt helper functions ở top-level
   - Sử dụng docstrings cho tasks
   - Group related tasks với TaskGroups

### ⚠️ Common Pitfalls:

1. **Import errors**: Functions phải import được từ DAG file
2. **XCom size limits**: Không pass data quá lớn qua XCom
3. **Connection strings**: Không hardcode trong code
4. **Timezone issues**: Luôn sử dụng UTC và pendulum
5. **Task dependencies**: Đảm bảo dependencies đúng


## 9. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. BashOperator - Chạy shell commands
2. PythonOperator - Chạy Python functions
3. SQLExecuteQueryOperator - Thực thi SQL queries
4. Hooks - Kết nối với external systems
5. So sánh @task decorator vs Operators
6. Tạo pipeline với nhiều operators
7. Best practices và common pitfalls

### 📚 Next Lab:
- **Lab 4**: Task Dependencies và Branching
- Bitshift operators (>>, <<)
- BranchPythonOperator
- Trigger rules
- Dynamic task mapping

### 🔗 Useful Links:
- [Airflow Operators](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-operators/index.html)
- [Airflow Hooks](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/hooks.html)
- [SQL Operators](https://airflow.apache.org/docs/apache-airflow-providers-common-sql/stable/operators/sql.html)
- [Best Practices](https://airflow.apache.org/docs/apache-airflow/3.1.1/best-practices/index.html)

### 💡 Exercises:
1. Tạo DAG với BashOperator để process files
2. Tạo DAG với PythonOperator để tính toán statistics
3. Tạo DAG với SQLExecuteQueryOperator để query database
4. Kết hợp tất cả operators trong một pipeline
